In [1]:
import re
from functools import reduce
import itertools
from collections import Counter
import numpy as np


def get_input(n):
    with open('input_'+n+'.txt', 'r') as infile:
        return infile.read().strip()


def parse_line(line):
    springs, numbers = line.strip().split()
    numbers = list(map(int, numbers.split(',')))
    return springs, numbers

def parse_input(puzzle,line_parser=parse_line):
    lines = puzzle.strip().split('\n')
    for line in lines:
        yield(line_parser(line))

def arrangement_candidates(springs, missing_springs):
    unknowns = list(re.finditer(r'\?',springs))
    if len(unknowns) < missing_springs:
        raise Exception("Too few unknowns impossible to solve")
    indices = [u.start() for u in unknowns]
    combinations = list(itertools.combinations(indices, missing_springs))
    for c in combinations:
        s = [s if i not in c else '#' for i, s in enumerate(springs)]
        yield ''.join(s)

assert list(arrangement_candidates('???',1)) == ['#??', '?#?', '??#']
assert list(arrangement_candidates('???',2)) == ['##?','#?#', '?##'], f"got {list(arrangement_candidates('???',2))} instead"
assert list(arrangement_candidates('???',3)) == ['###']

def is_correct(springs,numbers):
    matches = re.findall('(#+)', springs)
    if not matches or len(matches) != len(numbers):
        return False
    for spring, length in zip(matches, numbers):
        if len(spring) != length:
            return False
    return True

assert is_correct(*parse_line('###..###.# 3,3,1'))
assert not is_correct(*parse_line('#.#..###.# 3,3,1'))
print(parse_line('###..###.# 3,3,1'))


sample1 = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""

n_arrangements = [
    ["???.### 1,1,3", 1],
    [".??..??...?##. 1,1,3", 4],
    ["?#?#?#?#?#?#?#? 1,3,1,6", 1],
    ["????.#...#... 4,1,1", 1],
    ["????.######..#####. 1,6,5", 4],
    ["?###???????? 3,2,1", 10]
]

clean_sample = """#.#.### 1,1,3
.#...#....###. 1,1,3
.#.###.#.###### 1,3,1,6
####.#...#... 4,1,1
#....######..#####. 1,6,5
.###.##....# 3,2,1"""

valid_lines = parse_input(clean_sample)
assert all([is_correct(*x) for x in valid_lines])

def n_solutions(springs,numbers):
    try:
        missing = sum(numbers)-Counter(springs).get('#',0)

    except:
        print(springs,numbers)
        return 1
    return sum( is_correct(c,numbers) for c in arrangement_candidates(springs,missing) )

for line, n in n_arrangements:
    springs, numbers = parse_line(line)
    assert n_solutions(springs,numbers) == n


('###..###.#', [3, 3, 1])


In [2]:
puzzle = get_input('12')

In [3]:
def solve(puzzle):
    return sum(n_solutions(s,n) for s,n in parse_input(puzzle))
solve(sample1)

21

In [4]:
solve(puzzle)

7407

In [5]:
def parse_line2(line):
    springs, numbers = line.strip().split()
    numbers = list(map(int, numbers.split(',')))
    return '?'.join([springs]*5), numbers*5

In [18]:
cache = {}


def dp_n_solutions(springs, numbers, si, ni, current):
    key = (si, ni, current)
    if key == (0, 0, 0):
        pass
        cache.clear()
    if key in cache:
        pass
        return cache[key]
    if si == len(springs):
        if ni == len(numbers) and current == 0:
            return 1
        if ni == len(numbers)-1 and current == numbers[ni]:
            return 1
        return 0
    # Iteration:
    ans = 0
    for c in ['#', '.']:
        if springs[si] in ['?', c]:
            if c == '#':
                ans += dp_n_solutions(springs, numbers, si+1, ni, current+1)
            if c == '.':
                if current == 0:
                    ans += dp_n_solutions(springs, numbers, si+1, ni, current)
                if current > 0 and ni < len(numbers) and current == numbers[ni]:
                    ans += dp_n_solutions(springs, numbers, si+1, ni+1, 0)
    cache[key] = ans
    return ans


assert dp_n_solutions('###', [1, 2], 0, 0, 0) == 0
assert dp_n_solutions('###', [3], 0, 0, 0) == 1
dp_n_solutions(springs, numbers, 0, 0, 0)
dp_n_solutions(springs, numbers, 0, 0, 0)

for line, n in n_arrangements:
    springs, numbers = parse_line(line)
    print(f'validating against {line} solution: {n}')
    solution = dp_n_solutions(springs, numbers, 0, 0, 0)
    assert solution == n, f"expected {n} got {solution}"


def solve1_dp(puzzle):
    return sum(dp_n_solutions(s, n, 0, 0, 0) for s, n in parse_input(puzzle))

def solve2_dp(puzzle):
    return sum(dp_n_solutions(s, n, 0, 0, 0) for s, n in parse_input(puzzle, line_parser=parse_line2))

validating against ???.### 1,1,3 solution: 1
validating against .??..??...?##. 1,1,3 solution: 4
validating against ?#?#?#?#?#?#?#? 1,3,1,6 solution: 1
validating against ????.#...#... 4,1,1 solution: 1
validating against ????.######..#####. 1,6,5 solution: 4
validating against ?###???????? 3,2,1 solution: 10


In [11]:
solve1_dp(sample1)

21

In [14]:
solve1_dp(puzzle)

7407

In [19]:
solve2_dp(puzzle)

30568243604962